# Reconnaissance de chiffres manuscrits : MNIST

## Librairies et fonctions utiles

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split

from IPython.core.display import HTML # permet d'afficher du code html dans jupyter

## Le dataset de chiffres manuscrits MNIST

On charge le dataset MNIST :

In [ ]:
df = pd.read_csv("../input/mnist-in-csv/mnist_test.csv")

In [ ]:
df.head()

In [ ]:
df.shape

On a 785 colonnes :
* une colonne 'label' identifiant le chiffre  
* et 784 colonnes de pixels (image de 28x28 pixels "aplatie")

In [ ]:
df.head(10)

On crée la cible y (colonne 'label') :

In [ ]:
y = df['label']

et les caractéristiques X :

In [ ]:
X = df.drop(['label'], axis=1)

On sépare les ensembles d'apprentissage et de test :

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

On peut maintenant appliquer les méthodes de machine learning, mais auparavant on va visualiser les images

## Visualisation des images MNIST

Pour visualiser les images, on va convertir une ligne de 784 pixels en une matrice 28x28  
Il faut en premier transformer le dataframe X en un tableau :

In [ ]:
X1 = np.array(X)

On affiche la première ligne :

In [ ]:
print(X1[0])

On applique la méthode **reshape** pour convertir cette ligne de 784 éléments en une matrice 28x28 :

In [ ]:
image = X1[0].reshape(28,28)
print(image)

On peut maintenant afficher cette matrice :

In [ ]:
plt.imshow(image)

en niveaux de gris, sans graduation des axes, et avec le label comme titre :

In [ ]:
plt.imshow(image, cmap="gray_r")
plt.axis('off')
plt.title(y[0])

On redimensionne toutes les lignes :

In [ ]:
n_samples = len(df.index)
images = X1.reshape(n_samples,28,28)

On affiche les 50 premiers :

In [ ]:
plt.figure(figsize=(10,20))
for i in range(0,49) :
    plt.subplot(10,5,i+1)
    plt.axis('off')
    plt.imshow(images[i], cmap="gray_r")
    plt.title(y[i])

## Machine learning

Appliquer des méthodes de machine learning et évaluer les résultats (accuracy, matrice de confusion, ...)

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
rf_score = accuracy_score(y_test, y_rf)
print(rf_score)

In [ ]:
pd.crosstab(y_test, y_rf, rownames=['Reel'], colnames=['Prediction'], margins=True)

Vous pouvez également si vous le souhaitez tester l'algorithme XGBoost, souvent très efficace :  
https://datascientest.com/xgboost-grand-gagnant-des-competitions-machine-learning-algorithme  
https://medium.com/sfu-cspmp/xgboost-a-deep-dive-into-boosting-f06c9c41349

In [ ]:
import xgboost as xgb

In [ ]:
# XGB Classifier
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=40, seed=123)
eval_set = [(X_train, y_train), (X_test, y_test)]

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train, eval_metric=['merror'], eval_set=eval_set, verbose=True)
results = xg_cl.evals_result()

# Predict the labels of the test set: preds
predictions = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(predictions == y_test))/y_test.shape[0]
print("accuracy: %f" % (accuracy*100))

# plot classification error
epochs = len(results['validation_0']['merror'])
x_axis = range(0, epochs)
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['merror'], label='Train')
ax.plot(x_axis, results['validation_1']['merror'], label='Test')